# 推荐系统实例一
参考：项亮 《推荐系统实践》
## 推荐系统评测

什么才是好的推荐系统？这是推荐系统评测需要解决的首要问题。一个完整的推荐系统一般存在3个参与方：用户、物品提供者和提供推荐系统的网站。在评测一个推荐算法时，需要同时考虑三方的利益，一个好的推荐系统是能够令三方共赢的系统。

## 评测指标

推荐系统的评测指标用于评价推荐系统各方面的性能。这些指标有些可以定量计算，有些只能定性描述，有些可以通过离线实验计算，有些需要通过用户调查获得，还有些只能在线评测。对于重要的评测指标，后面几章将会详细讨论如何优化它们。下面将详细讨论各个不同的指标。

**1. 用户满意度** 

用户作为推荐系统的重要参与者，其满意度是评测推荐系统的最重要指标。但是，用户满意度没有办法离线计算，只能通过用户调查或者在线实验获得。

**2. 预测准确度** 

预测准确度度量一个推荐系统或者推荐算法预测用户行为的能力。这个指标是最重要的推荐系统离线评测指标，从推荐系统诞生的那一天起，几乎99%与推荐相关的论文都在讨论这个指标。这主要是因为该指标可以通过离线实验计算，方便了很多学术界的研究人员研究推荐算法。

在计算该指标时需要有一个离线的数据集，该数据集包含用户的历史行为记录。然后，将该数据集通过时间分成训练集和测试集。最后，通过在训练集上建立用 户的行为和兴趣模型预测用户在测试集上的行为，并计算预测行为和测试集上实际行为的重合度作为预测准确度。

评分预测 很多提供推荐服务的网站都有一个让用户给物品打分的功能。知道了用户对物品的历史评分，就可以从中习得用户的兴趣模型，并预测该用户在将来看到一个他没有评过分的物品时，会给这个物品评多少分。预测用户对物品评分的行为称为评分预测。
     
评分预测的预测准确度一般通过**均方根误差（RMSE）和平均绝对误差（MAE）**计算。对于测试集中的一个用户u和物品i，令$r_{ui}$是用户u对物品i的实际评分，而$\hat r_{ui}$是推荐算法给出的预测评分，那么**RMSE**的定义为：$$RMSE=\frac{\sqrt{\sum_{u,i\in T}(r_{ui}-\hat r_{ui})^2}}{|T|}$$

**MAE采用绝对值计算预测误差**，它的定义为：$$MAE=\frac{\sum_{u,i\in T}|r_{ui}-\hat r_{ui}|}{|T|}$$

假设我们用一个列表records存放用户评分数据，令records[i] = [u,i,rui,pui]，其中rui是用户u对物品i的实际评分，pui是算法预测出来的用户u对物品i的评分，那么下面的代码分别实现了RMSE和MAE的计算过程。 

In [ ]:
def RMSE(records): 
    return math.sqrt(sum([(rui-pui)*(rui- pui) for u,i,rui,pui in records]) / float(len(records))) 
def MAE(records): 
    return sum([abs(rui- pui) for u,i,rui,pui in records]) / float(len(records)) 


关于RMSE和MAE这两个指标的**优缺点**， Netflix认为RMSE加大了对预测不准的用户物品评分的惩罚（平方项的惩罚），因而对系统的评测更加苛刻。研究表明，如果评分系统是基于整数建立的（即用户给的评分都是整数），那么对预测结果取整会降低MAE的误差。

网站在提供推荐服务时，一般是给用户一个个性化的推荐列表，这种推荐叫做**TopN推荐**。TopN推荐的预测准确率一般通过**准确率 （precision）/召回率（recall）度量**。 令R(u)是根据用户在训练集上的行为给用户作出的推荐列表，而T(u)是用户在测试集上的行为列表。那么，推荐结果的**召回率定义**为：$$Recall=\frac{\sum_{u\in U}|R(u)\cap T(u)|}{\sum_{u\in U}|T(u)|}$$

推荐结果的准确率定义为$$Precision=\frac{\sum_{u\in U}|R(u)\cap T(u)|}{\sum_{u\in U}|R(u)|}$$下面的Python代码同时计算出了一个推荐算法的准确率和召回率：

**3. 覆盖率**

覆盖率（coverage）描述一个推荐系统对物品长尾的发掘能力。覆盖率有不同的 定义方法，最简单的定义为推荐系统能够推荐出来的物品占总物品集合的比例。假设系统的用户集合为U，推荐系统给每个用户推荐一个长度为N的物品列表R(u)。那么推荐系统的覆盖率可以通过下面的公式计算：$$Coverage=\frac{|\bigcup _{u\in U}R(u)|}{|I|}$$

**4. 多样性**

**5. 新颖性**

新颖的推荐是指给用户推荐那些他们以前没有听说过的物品。在一个网站中实现 新颖性的最简单办法是，把那些用户之前在网站中对其有过行为的物品从推荐列 表中过滤掉。比如在一个视频网站中，新颖的推荐不应该给用户推荐那些他们已 经看过、打过分或者浏览过的视频。但是，有些视频可能是用户在别的网站看 过，或者是在电视上看过，因此仅仅过滤掉本网站中用户有过行为的物品还不能 完全实现新颖性。

**6. 惊喜度**

惊喜度（serendipity）是最近这几年推荐系统领域最热门的话题。但什么是惊喜 度，惊喜度与新颖性有什么区别是首先需要弄清楚的问题。注意，这里讨论的是 惊喜度和新颖度作为推荐指标在意义上的区别，而不是这两个词在中文里的含义 区别（因为这两个词是英文词翻译过来的，所以它们在中文里的含义区别和英文 词的含义区别并不相同），所以我们首先要摒弃大脑中关于这两个词在中文中的 基本含义。

**7. 信任度**

如果你有两个朋友，一个人你很信任，一个人经常满嘴跑火车，那么如果你信任 的朋友推荐你去某个地方旅游，你很有可能听从他的推荐，但如果是那位满嘴跑 火车的朋友推荐你去同样的地方旅游，你很有可能不去。这两个人可以看做两个 推荐系统，尽管他们的推荐结果相同，但用户却可能产生不同的反应，这就是因 为用户对他们有不同的信任度。

**8. 实时性**

在很多网站中，因为物品（新闻、微博等）具有很强的时效性，所以需要在物品 还具有时效性时就将它们推荐给用户。比如，给用户推荐昨天的新闻显然不如给 用户推荐今天的新闻。因此，在这些网站中，推荐系统的实时性就显得至关重 要。

**9. 健壮性**

任何一个能带来利益的算法系统都会被人攻击，这方面最典型的例子就是搜索引 擎。搜索引擎的作弊和反作弊斗争异常激烈，这是因为如果能让自己的商品成为 热门搜索词的第一个搜索果，会带来极大的商业利益。推荐系统目前也遇到了同 样的作弊问题，而健壮性（即robust,鲁棒性）指标衡量了一个推荐系统抗击作弊 的能力。

# 利用用户行为数据
## 实验设计与算法评测
### 数据集介绍
参考：https://grouplens.org/datasets/

网站中GroupLens提供了**MovieLens数据集**。包含了6000多用户对4000部电影的100万条评价，该数据集是一个**评分数据集**（具有历史意义的推荐系统竞赛所用的数据集），用户可以给电影评5个不同等级的分数（1～5 分）。本章着重研究隐反馈数据集中的TopN推荐问题，因此忽略了数据集中的评分记录。也就是说，TopN推荐的任务是预测用户会不会对某部电影评分，而不是预测用户在准备对某部电影评分的前提下会给电影评多少分。

数据集它分为**三个表**：评分、用户信息和电影信息。这些数据都是dat文件格式，可以通过pandas模块来处理。

**ratings数据**

文件里面的内容包含了每一个用户对于每一部电影的评分。数据格式如下：userId, movieId, rating, timestamp

userId: 每个用户的id

movieId: 每部电影的id

rating: 用户评分，是5星制，按半颗星的规模递增(0.5 stars - 5 stars)

timestamp: 自1970年1月1日零点后到用户提交评价的时间的秒数

数据排序的顺序按照userId，movieId排列的。

**movies数据**

文件里包含了一部电影的id和标题，以及该电影的类别。数据格式如下：movieId, title, genres

movieId:每部电影的id

title:电影的标题

genres:电影的类别。电影分类是以竖线|分割的字符串形式给出的。如果想对不同的电影分类进行分析的话，就需要先将其转换成更有用的形式才行（详细分类见readme.txt）

**users数据**

包含了用户信息：UserID，Gender，Age，Occupation，Zip-code

UserID：用户的ID

Gender：性别

Age：年龄

Occupation：职业

Zip-code：用户的邮政编码

年龄和职业是以**编码形式**给出的，它们的具体含义请参考该数据集的REAMDE文件。

### Pandas简介
Python Data Analysis Library 或 pandas 是基于NumPy 的一种工具，该工具是为了解决数据分析任务而创建的。Pandas 纳入了大量库和一些标准的数据模型，提供了高效地操作大型数据集所需的工具和快速便捷地处理数据的函数和方法。

**Pandas**是python的一个数据分析包，最初由AQR Capital Management于2008年4月开发，并于2009年底开源出来，目前由专注于Python数据包开发的PyData开发team继续开发和维护，属于PyData项目的一部分。Pandas最初被作为金融数据分析工具而开发出来，因此，pandas为时间序列分析提供了很好的支持。 Pandas的名称来自于面板数据（panel data）和python数据分析（data analysis）。panel data是经济学中关于多维数据集的一个术语，在Pandas中也提供了panel的数据类型。

**数据结构：**

**Series**：一维数组，与Numpy中的一维array类似。二者与Python基本的数据结构List也很相近，其区别是：List中的元素可以是不同的数据类型，而Array和Series中则只允许存储相同的数据类型，这样可以更有效的使用内存，提高运算效率。

**Time- Series**：以时间为索引的Series。

***DataFrame**：二维的表格型数据结构。很多功能与R中的data.frame类似。可以将DataFrame理解为Series的容器。以下的内容主要以DataFrame为主。

**Panel** ：三维的数组，可以理解为DataFrame的容器。

Pandas 有两种自己独有的基本数据结构。读者应该注意的是，它固然有着两种数据结构，因为它依然是 Python 的一个库，所以，Python 中有的数据类型在这里依然适用，也同样还可以使用类自己定义数据类型。只不过，Pandas 里面又定义了两种数据类型：**Series 和 DataFrame**，它们让数据操作更简单了。

因为pandas是python的第三方库，所以使用前需要安装一下，直接使用**pip install pandas** 就会自动安装pandas以及相关组件。

In [ ]:
from pandas import Series
import pandas as pd
s = Series([1,4,'ww','tt'])
s

Series 就如同列表一样，一系列数据，每个数据对应一个索引值。这里，我们实质上创建了一个 Series 对象，这个对象当然就有其属性和方法了。比如，下面的两个**属性**依次可以显示 Series 对象的数据值和索引：

In [ ]:
s.index

In [ ]:
s.values

In [ ]:
#索引只能是从 0 开始的整数。Series 可以自定义索引
#如果需要定义 index，放在后面
s2 = Series(['wangxing','man',24],index=['name','sex','age'])
s2

In [ ]:
s2['name']#有点类似dict

**DataFrame 是一种二维的数据结构**，非常接近于电子表格或者类似 mysql 数据库的形式。它的竖行称之为 columns，横行跟前面的 Series 一样，称之为 index，也就是说可以通过 columns 和 index 来确定一个主句的位置。

In [ ]:
from pandas import Series,DataFrame
data = {"name":['google','baidu','yahoo'],"marks":[100,200,300],"price":[1,2,3]}
f1 = DataFrame(data)
f1

In [ ]:
#可以规定列的顺序
f2 = DataFrame(data,columns=['name','price','marks'])
f2

### 数据集示例

In [ ]:
import pandas as pd

# 用户信息
unames = ['user_id', 'gender', 'age', 'occupation', 'zip-code']
users = pd.read_table('Recomment_System/users.dat', sep='::', header=None, names=unames, engine='python')

# 电影排名
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('Recomment_System/ratings.dat', sep='::', header=None, names=rnames,engine='python')

# 电影信息
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('Recomment_System/movies.dat', sep='::', header=None, names=mnames, engine='python')


In [ ]:
users.head()

In [ ]:
users[0:5]

In [ ]:
ratings[0:5]

In [ ]:
movies[0:5]

将所有数据都合并到一个表中，pandas会根据列名的重叠情况推断出哪些列是合并（或连接）键。

In [ ]:
data = pd.merge(pd.merge(ratings, users), movies)
data

按性别计算每部电影的平均得分，我们可以使用pivot_table方法(也可以用groundby对'title'和'gendef'分组之后unstack可以得到相同的结果)

In [ ]:
mean_ratings = data.pivot_table('rating', index='title', columns='gender', aggfunc='mean')
mean_ratings[0:5]

### 实验设计 
协同过滤算法的离线实验一般如下设计。首先，将用户行为数据集按照均匀分布随机分成M 份（本章取M=8），挑选一份作为测试集，将剩下的M-1份作为训练集。然后在训练集上建 立用户兴趣模型，并在测试集上对用户行为进行预测，统计出相应的评测指标。为了保证评 测指标并不是过拟合的结果，需要进行M次实验，并且每次都使用不同的测试集。然后将M次实验测出的评测指标的**平均值**作为最终的评测指标。 

下面的Python代码描述了将数据集随机分成训练集和测试集的过程：

In [ ]:
import random
import math
from operator import itemgetter

In [ ]:
def SplitData(data, M, k, seed): 
    test = [] 
    train = [] 
    random.seed(seed) 
    for user, item in data: 
        if random.randint(0,M) == k: 
            test.append([user,item]) 
        else:
            train.append([user,item]) 
    return train, test

这里，每次实验选取不同的k（0≤k≤M-1）和相同的随机数种子seed，进行M次实验就可以 得到M个不同的训练集和测试集，然后分别进行实验，用M次实验的平均值作为最后的评测 指标。这样做主要是防止某次实验的结果是过拟合的结果（over fitting），但如果数据集够 大，模型够简单，为了快速通过离线实验初步地选择算法，也可以只进行一次实验。

### 评测指标
通过**准确率/召回率**评测推荐算法的精度。**召回率**描述有多少比例的用户—物品评分记录包含在最终的推荐列表中，而**准确率**描述最终的推荐列表中有多少比例是发生过的用户—物品评分记录。

下面两段代码给出了召回率和准 确率的计算方法。

In [ ]:
def Recall(train, test, N): 
    hit = 0 
    all = 0 
    for user in train.keys(): 
        tu = test[user] 
        rank = GetRecommendation(user, N) 
        for item, pui in rank: 
            if item in tu: 
                hit += 1 
            all += len(tu) 
            return hit / (all * 1.0) 
        
def Precision(train, test, N): 
    hit = 0 
    all = 0 
    for user in train.keys(): 
        tu = test[user] 
        rank = GetRecommendation(user, N) 
        for item, pui in rank:
            if item in tu: 
                hit += 1 
        all += N 
    return hit / (all * 1.0)

**覆盖率**反映了推荐算法发掘长尾的 能力，覆盖率越高，说明推荐算法越能够将长尾中的物品推荐给用户。

这里，我们采用最简单的覆盖率定义。该覆盖率表示最终的推荐列表中包含多大比例的物品。如果所有的物品都被推荐给至少一个 用户，那么覆盖率就是100%。如下代码可以用来计算推荐算法的覆盖率：

In [ ]:
def Coverage(train, test, N): 
    recommend_items = set() 
    all_items = set() 
    for user in train.keys(): 
        for item in train[user].keys(): 
            all_items.add(item) 
        rank = GetRecommendation(user, N) 
        for item, pui in rank: 
            recommend_items.add(item) 
    return len(recommend_items) / (len(all_items) * 1.0)

最后，我们还需要评测推荐的**新颖度**，这里用推荐列表中物品的平均流行度度量推荐结果的新颖度。如果推荐出的物品都很热门，说明推荐的新颖度较低，否则说明推荐结果比较新颖。

In [ ]:
def Popularity(train, test, N): 
    item_popularity = dict() 
    for user, items in train.items(): 
        for item in items.keys() :
            if item not in item_popularity: 
                item_popularity[item] = 0 
            item_popularity[item] += 1 
    ret = 0 
    n = 0 
    for user in train.keys(): 
        rank = GetRecommendation(user, N) 
        for item, pui in rank: 
            ret += math.log(1 + item_popularity[item])
            n += 1 
        ret /= n * 1.0 
    return ret 

这里，在计算平均流行度时对每个物品的流行度取对数，这是因为物品的流行度分布满足长 尾分布，在取对数后，流行度的平均值更加稳定。

## 基于邻域的算法
基于邻域的算法是推荐系统中最基本的算法，该算法不仅在学术界得到了深入研究，而且在业界得到了广泛应用。**基于邻域的算法分为两大类**，一类是**基于用户的协同过滤算法**，另一 类是**基于物品的协同过滤算法**。下面几节将对这两种算法进行深入介绍，对比它们的优缺点 并提出改进方案。
### 基于用户的协同过滤算法 
基于用户的协同过滤算法**UserCF**是推荐系统中最古老的算法。可以不夸张地说，这个算法的诞生标志了推荐系统的诞生。该算法在1992年被提出，并应用于邮件过滤系统，1994年被 GroupLens用于新闻过滤。在此之后直到2000年，该算法都是推荐系统领域最著名的算法。本节将对该算法进行详细介绍，首先介绍最基础的算法，然后在此基础上提出不同的**改进方法**，并通过真实的数据集进行评测。

#### 基础算法

在一个在线个性化推荐系统中，当一个用户A需要个性化推荐时，可以先找到**和他有相似兴趣的其他用户**，然后把那些用户喜欢的、而用户A没有听说过的物品推荐给A。这种方法称为基于用户的协同过滤算法。 

基于用户的协同过滤算法主要包括**两个步骤**。

1）找到和目标用户兴趣相似的用户集合。 

2）找到这个集合中的用户喜欢的，且目标用户没有听说过的物品推荐给目标用户。 

步骤(1)的关键就是计算两个用户的**兴趣相似度**。这里，协同过滤算法**主要利用行为的相似度计算兴趣的相似度**。给定用户u和用户v，令N(u)表示用户u曾经有过正反馈的物品集合，令 N(v)为用户v曾经有过正反馈的物品集合。那么，我们可以通过如下的Jaccard公式简单地计算u和v的兴趣相似度：$$w_{uv}=\frac{|N(u)\cap N(v)|}{|N(u)\cup N(v)|}$$

或者通过余弦相似度计算：$$w_{uv}=\frac{|N(u)\cap N(v)|}{\sqrt{|N(u)|| N(v)|}}$$下面举例说明UserCF计算用户兴趣相似度的例子。![jupyter](./recommend-3.png)在该例中，用户A对物品{a, b, d}有过行为，用户B对物品{a, c}有过行为，利用余弦相似度公式计算 用户A和用户B的兴趣相似度为：![jupyter](./recommend-1.png)以余弦相似度为例，实现该相似度可以利用如下的伪码：

用户A和用户C的兴趣相似度 ![jupyter](./recommend-4.png) 
用户A和用户D的兴趣相似度 ![jupyter](./recommend-5.png)

In [ ]:
def UserSimilarity(train): 
    W = dict() 
    for u in train.keys(): 
        for v in train.keys(): 
            if u == v: 
                continue 
            W[u][v] = len(train[u] & train[v])#??? 
            W[u][v] /= math.sqrt(len(train[u]) * len(train[v]) * 1.0) 
    return W

该代码对两两用户都利用余弦相似度计算相似度。这种方法的时间复杂度是$O(|U|*|U|)$，这在用户数很大时非常**耗时**。事实上，很多用户相互之间并没有对同样的物品产生过行为，即很多时候$|N(u)\cap N(v)|=0$。上面的算法将很多时间浪费在了计算这种用户之间的相似度上。如果换一个思路，我们可以首先计算出$|N(u)\cap N(v)|\neq 0$ 的用户对$(u, v)$，然后再对这种情况除以分母$\sqrt{|N(u)||N(v)|}$ 。

为此，可以首先建立物品到用户的**倒查表**，对于每个物品都保存对该物品产生过行为的用户列表。![jupyter](./recommend-6.png)图右上为到查表，下方为稀疏矩阵$C[u][v]=|N(u)\cap N(v)| $。那么，假设用户u和用户v同时属于倒查表中K个物品对应的用户列表，就有$C [u][v]=K$。从而，可以扫描倒查表中每个物品 对应的用户列表，将用户列表中的两两用户对应的$C [u][v]$加1，最终就可以得到所有用户之间不为0的$C [u][v]$。

对于物品 a 将W[A][B] 和W[B][A]都加1 对于物品b将W[A][C]和W[C][A]都加1 等等。。。 
假如 用户A 喜欢{a,b,d} 用户B喜欢{a,b,c} 则倒排表为 

a A B 

b A B C 

则在计算的时候 W[A][B] 和 W[B][A]都为2 这样对于余弦相似度公式的分子就有了 再除以分母就行了。

下面的代码实现了上面提到的算法：

In [ ]:
def UserSimilarity(train): 
    # build inverse table for item_users 
    item_users = dict()
    for u, items in train.items(): 
        for i in items.keys(): 
            if i not in item_users: 
                item_users[i] = set() 
            item_users[i].add(u) 
    #calculate co-rated items between users 
    C = dict() 
    N = dict() 
    for i, users in item_users.items(): 
        for u in users:
            N[u] += 1 
            for v in users: 
                if u == v: 
                    continue 
        C[u][v] += 1 
    #calculate finial similarity matrix W 
    W = dict() 
    for u, related_users in C.items(): 
        for v, cuv in related_users.items(): 
             W[u][v] = cuv / math.sqrt(N[u] * N[v]) 
    return W

![jupyter](./recommend-1.png)用图例解释上面的算法。首先，需要建立物品—用户的倒排表。然后，建立一个4×4的用户相似度矩阵W，对于物品a，将W [A][B]和W [B][A]加 1，对于物品b，将W [A][C]和W [C][A]加1，以此类推。扫描完所有物品后，我们可以得到最 终的W矩阵。这里的W是余弦相似度中的分子部分，然后将W除以分母可以得到最终的用户兴趣相似度。

得到用户之间的兴趣相似度后，UserCF算法会给用户推荐和他兴趣最相似的K个用户喜欢的物品。如下的公式度量了UserCF算法中**用户u对物品i的感兴趣程度**：$$p(u,i)=\sum_{v\in S(u,K) \bigcap N(i)}w_{uv}r_{vi}$$

$S(u,K)$是和用户u兴趣最接近的K个用户（K近邻），$N(i)$是对物品i行为的用户的集合，$w_{uv}$是用户u和用户v的兴趣相似度，$r_{vi}$代表用户v对物品i的兴趣，因为使用的是单一行为的隐反馈数据，所以所有的$r_{vi}=1$。

如下代码实现了上面的UserCF推荐算法：

In [ ]:
def Recommend(user, train, W): 
    rank = dict() 
    interacted_items = train[user] 
    for v, wuv in sorted(W[u].items, key=itemgetter(1),reverse=True)[0:K]: 
        for i, rvi in train[v].items: 
            if i in interacted_items: 
                #we should filter items user interacted before 
                continue 
            rank[i] += wuv * rvi 
    return rank 

#### 用户相似度计算的改进

上一节介绍了计算用户兴趣相似度的最简单的公式（余弦相似度公式），但这个公式过于粗 糙，本节将讨论如何改进该公式来提高UserCF的推荐性能。 

首先，以图书为例，如果两个用户都曾经买过《新华字典》，这丝毫不能说明他们兴趣相 似，因为绝大多数中国人小时候都买过《新华字典》。但如果两个用户都买过《数据挖掘导 论》，那可以认为他们的兴趣比较相似，因为只有研究数据挖掘的人才会买这本书。换句话说，**两个用户对冷门物品采取过同样的行为更能说明他们兴趣的相似度**。因此，John S. Breese提出了如下公式，根据用户行为计算用户的兴趣相似度：$$w_{uv}=\frac{\sum_{i\in N(u)\bigcap N(v)}\frac{1}{log(1+|N(i))|}}{\sqrt{|N(u)||N(v)|}}$$

可以看到，该公式通过$\frac{1}{log(1+|N(i))|}$惩罚了用户u和用户v共同兴趣列表中热门物品对他们相似度的影响。

In [ ]:
def UserSimilarity(train): 
    # build inverse table for item_users 
    item_users = dict() 
    for u, items in train.items(): 
        for i in items.keys(): 
            if i not in item_users: 
                item_users[i] = set() 
            item_users[i].add(u)
#calculate co-rated items between users 
    C = dict() 
    N = dict() 
    for i, users in item_users.items(): 
        for u in users: 
            N[u] += 1 
            for v in users: 
                if u == v: 
                    continue 
                C[u][v] += 1 / math.log(1 + len(users)) 
    #calculate finial similarity matrix W 
    W = dict() 
    for u, related_users in C.items(): 
        for v, cuv in related_users.items(): 
            W[u][v] = cuv / math.sqrt(N[u] * N[v]) 
    return W 

#### 基于用户的协同过滤算法代码实现

In [ ]:
#-*- coding: utf-8 -*-
'''
Created on 2015-06-22

@author: Lockvictor
'''
import sys
import random
import math
import os
from operator import itemgetter

from collections import defaultdict

random.seed(0)

'''
users.dat 数据集 
用户id 用户性别 用户年龄 用户职业 用户所在地邮编
1::F::1::10::48067
2::M::56::16::70072
3::M::25::15::55117

movies.dat 数据集
电影id 电影名称 电影类型 
250::Heavyweights (1994)::Children's|Comedy
251::Hunted, The (1995)::Action
252::I.Q. (1994)::Comedy|Romance

ratings.dat 数据集
用户id 电影id 用户评分  时间戳
157::3519::4::1034355415
157::2571::5::977247494
157::300::3::977248224

'''

class UserBasedCF(object):
    ''' TopN recommendation - User Based Collaborative Filtering '''

    # 构造函数，用来初始化
    def __init__(self):
        # 定义 训练集 测试集 为字典类型
        self.trainset = {}
        self.testset = {}
        # 训练集用的相似用户数
        self.n_sim_user = 20
        # 推荐电影数量
        self.n_rec_movie = 10

        self.user_sim_mat = {}
        self.movie_popular = {}
        self.movie_count = 0
        # sys.stderr 是用来重定向标准错误信息的
        print ('相似用户数目为 = %d' % self.n_sim_user, file=sys.stderr)
        print ('推荐电影数目为 = %d' % self.n_rec_movie, file=sys.stderr)


    # 加载文件
    @staticmethod
    def loadfile(filename):
        ''' load a file, return a generator. '''
        # 以只读的方式打开传入的文件
        fp = open(filename, 'r')
        # enumerate()为枚举，i为行号从0开始，line为值
        for i, line in enumerate(fp):
            # yield 迭代去下一个值，类似next()
                # line.strip()用于去除字符串头尾指定的字符。
            yield line.strip('\r\n')
            # 计数
            if i % 100000 == 0:
                print ('loading %s(%s)' % (filename, i), file=sys.stderr)
        fp.close()
        # 打印加载文件成功
        print ('load %s succ' % filename, file=sys.stderr)

    # 划分训练集和测试集，pivot用来定义训练集和测试集的比例
    def generate_dataset(self, filename, pivot=0.7):
        ''' load rating data and split it to training set and test set '''
        trainset_len = 0
        testset_len = 0

        for line in self.loadfile(filename):
            # 根据 分隔符 :: 来切分每行数据
            user, movie, rating, _ = line.split('::')
            # 随机数字 如果小于0.7 则数据划分为训练集
            if random.random() < pivot:
                # 设置训练集字典，key为user,value 为字典 且初始为空
                self.trainset.setdefault(user, {})
                # 以下省略格式如下，集同一个用户id 会产生一个字典，且值为他评分过的所有电影
                #{'1': {'914': 3, '3408': 4, '150': 5, '1': 5}, '2': {'1357': 5}}
                self.trainset[user][movie] = int(rating)
                trainset_len += 1
            else:
                self.testset.setdefault(user, {})
                self.testset[user][movie] = int(rating)
                testset_len += 1
        # 输出切分训练集成功
        print ('划分数据为训练集和测试集成功！', file=sys.stderr)
        # 输出训练集比例
        print ('训练集数目 = %s' % trainset_len, file=sys.stderr)
        # 输出测试集比例
        print ('测试集数目 = %s' % testset_len, file=sys.stderr)
        
    # 建立物品-用户 倒排表
    def calc_user_sim(self):
        ''' calculate user similarity matrix '''
        # build inverse table for item-users
        # key=movieID, value=list of userIDs who have seen this movie
        print ('构建物品-用户倒排表中，请等待......', file=sys.stderr)
        movie2users = dict()

        # Python 字典(Dictionary) items() 函数以列表返回可遍历的(键, 值) 元组数组
        for user, movies in self.trainset.items():
            for movie in movies:
                # inverse table for item-users
                if movie not in movie2users:
                    # 根据电影id 构造set() 函数创建一个无序不重复元素集
                    movie2users[movie] = set()
                # 集合中值为用户id
                # 数值形如
                # {'914': {'1','6','10'}, '3408': {'1'} ......}
                movie2users[movie].add(user)
                # 记录电影的流行度
                if movie not in self.movie_popular:
                    self.movie_popular[movie] = 0
                self.movie_popular[movie] += 1
        print ('构建物品-用户倒排表成功', file=sys.stderr)

        # save the total movie number, which will be used in evaluation
        self.movie_count = len(movie2users)
        print ('总共被操作过的电影数目为 = %d' % self.movie_count, file=sys.stderr)

        # count co-rated items between users
        usersim_mat = self.user_sim_mat

        print ('building user co-rated movies matrix...', file=sys.stderr)
        # 令系数矩阵 C[u][v]表示N(u)∩N（v) ，假如用户u和用户v同时属于K个物品对应的用户列表，就有C[u][v]=K
        for movie, users in movie2users.items():
            for u in users:
                usersim_mat.setdefault(u, defaultdict(int))
                for v in users:
                    if u == v:
                        continue
                    usersim_mat[u][v] += 1
        print ('build user co-rated movies matrix succ', file=sys.stderr)

        # calculate similarity matrix
        print ('calculating user similarity matrix...', file=sys.stderr)
        simfactor_count = 0
        PRINT_STEP = 2000000
        # 循环遍历usersim_mat 根据余弦相似度公式计算出用户兴趣相似度
        for u, related_users in usersim_mat.items():
            for v, count in related_users.items():
                # 以下是公式计算过程
                usersim_mat[u][v] = count / math.sqrt(
                    len(self.trainset[u]) * len(self.trainset[v]))
                #计数 并没有什么卵用
                simfactor_count += 1
                if simfactor_count % PRINT_STEP == 0:
                    print ('calculating user similarity factor(%d)' %
                           simfactor_count, file=sys.stderr)

        print ('calculate user similarity matrix(similarity factor) succ',
               file=sys.stderr)
        print ('Total similarity factor number = %d' %
               simfactor_count, file=sys.stderr)
    # 根据用户给予推荐结果
    def recommend(self, user):
        '''定义给定K个相似用户和推荐N个电影'''
        K = self.n_sim_user
        N = self.n_rec_movie
        # 定义一个字典来存储为用户推荐的电影
        rank = dict()
        watched_movies = self.trainset[user]
        # sorted() 函数对所有可迭代的对象进行排序操作。 key 指定比较的对象 ，reverse=True 降序
        for similar_user, similarity_factor in sorted(self.user_sim_mat[user].items(),
                                                      key=itemgetter(1), reverse=True)[0:K]:
            for movie in self.trainset[similar_user]:
                # 判断 如果这个电影 该用户已经看过 则跳出循环
                if movie in watched_movies:
                    continue
                # 记录用户对推荐的电影的兴趣度
                rank.setdefault(movie, 0)
                rank[movie] += similarity_factor
        # return the N best movies
        return sorted(rank.items(), key=itemgetter(1), reverse=True)[0:N]

    # 计算 准确略，召回率，覆盖率，流行度
    def evaluate(self):

        ''' print evaluation result: precision, recall, coverage and popularity '''
        print ('Evaluation start...', file=sys.stderr)

        N = self.n_rec_movie
        #  varables for precision and recall
        #记录推荐正确的电影数
        hit = 0
        #记录推荐电影的总数
        rec_count = 0
        #记录测试数据中总数
        test_count = 0
        # varables for coverage
        all_rec_movies = set()
        # varables for popularity
        popular_sum = 0

        for i, user in enumerate(self.trainset):
            if i % 500 == 0:
                print ('recommended for %d users' % i, file=sys.stderr)
            test_movies = self.testset.get(user, {})
            rec_movies = self.recommend(user)
            for movie, _ in rec_movies:
                if movie in test_movies:
                    hit += 1
                all_rec_movies.add(movie)
                popular_sum += math.log(1 + self.movie_popular[movie])
            rec_count += N
            test_count += len(test_movies)
        # 计算准确度
        precision = hit / (1.0 * rec_count)
        # 计算召回率
        recall = hit / (1.0 * test_count)
        # 计算覆盖率
        coverage = len(all_rec_movies) / (1.0 * self.movie_count)
        #计算流行度
        popularity = popular_sum / (1.0 * rec_count)

        print ('precision=%.4f\trecall=%.4f\tcoverage=%.4f\tpopularity=%.4f' %
               (precision, recall, coverage, popularity), file=sys.stderr)


if __name__ == '__main__':
    ratingfile = os.path.join('Recomment_System', 'ratings.dat')
    usercf = UserBasedCF()
    usercf.generate_dataset(ratingfile)
    usercf.calc_user_sim()

    '''
    以下为用户id 为 1688的用户推荐的电影
    a = usercf.recommend("1688")
    [('1210', 3.1260082382168055), ('2355', 3.0990860017403934), ('1198', 2.692208437663706), ('1527', 2.643102457311887), ('3578', 2.61895974438311), ('1376', 2.469905776632142), ('110', 2.4324588006133383), ('1372', 2.4307454264036528), ('1240', 2.424265305355254), ('32', 2.3926144836965966)]
    '''
    usercf.evaluate()

### 基于物品的协同过滤算法 
Netflix Prize的最终获胜队伍通过**融合上百个模型**的结果才取得了最终的成功。由此可见**模型融合**对提高评分预测的精度至关重要。本节讨论模型融合的两种不同技术。

#### 模型级联融合
假设已经有一个预测器$\hat r^{(k)}$ ，对于每个用户—物品对$(u, i)$都给出预测值，那么可以在这个预测器的基础上设计下一个预测器 $\hat r^{(k+1)}$ 来最小化损失函数：$$C=\sum_{(u,i)\in Train}(r_{ui}-\hat r_{ui}^{(k)}-\hat r_{ui}^{(k+1)})^2$$ 由上面的描述可以发现，级联融合很像Adaboost算法。和**Adaboost算法**类似，该方法每次产生一个新模型，按照一定的参数加到旧模型上去，从而使训练集误差最小化。不同的是，这里每次生成新模型时并不对样本集采样，针对那些预测错的样本，而是每次都还是利用全样本集进行预测，但每次使用的模型都有区别。

一般来说，级联融合的方法都用于简单的预测器，比如前面提到的**平均值预测器**。即使是利用简单的算法进行级联融合，也能得到比较低的评分预测误差。下面的Python代码实现了利用平均值预测器进行级联融合的方法。

def Predict(Train,test,alpha)

#### 基础算法 
基于用户的协同过滤算法有一些**缺点**。**首先**，随着网站的用户数目越来越大，计算用户兴趣相似度矩阵将越来越困难，其运算时间复 杂度和空间复杂度的增长和用户数的增长近似于平方关系。**其次**，基于用户的协同过滤很难对推荐结果作出解释。因此，著名的电子商务公司亚马逊提出了另一个算法——基于物品的协同过滤算法

基于物品的协同过滤算法（**简称ItemCF**）给用户推荐那些和他们之前喜欢的物品相似的物 品。比如，该算法会因为你购买过《数据挖掘导论》而给你推荐《机器学习》。不过， ItemCF算法并不利用物品的内容属性计算物品之间的相似度，它主要通过分析用户的行为记录**计算物品之间的相似度**。该算法认为，物品A和物品B具有很大的相似度是因为喜欢物品A的用户大都也喜欢物品B。

基于物品的协同过滤算法主要分为**两步**。 

1. 计算物品之间的相似度。 

2. 根据物品的相似度和用户的历史行为给用户生成推荐列表。

（购买了该商品的用户也经常购买的其他商品）。从这句话的定义出发，我们可以用下面的公式定义**物品的相似度**：$$w_{ij}=\frac {|N(i)\cap N(j)|}{|N(i)|}$$$N(i)$为喜欢物品i的用户数，分子为同时喜欢物品i、j的用户数。

这里存在一个问题。如果物品j很热门，很多人都喜欢，那么$W_{ij}$就会很大，接近1。因此，该公式会造成任何物品都会和热门的物品有很大的相似度， 这对于致力于挖掘长尾信息的推荐系统来说显然不是一个好的特性。为了避免推荐出热门的物品，可以用下面的公式：$$w_{ij}=\frac {|N(i)\cap N(j)|}{\sqrt{|N(i)||N(j)|}}$$

这个公式惩罚了物品j的权重，因此减轻了热门物品会和很多物品相似的可能性。 

从上面的定义可以看到，在协同过滤中两个物品产生相似度是因为它们共同被很多用户喜欢，也就是说每个用户都可以通过他们的历史兴趣列表给物品“贡献”相似度。这里面蕴涵着一个假设，就是每个用户的兴趣都局限在某几个方面，因此如果两个物品属于一个用户的兴 趣列表，那么这两个物品可能就属于有限的几个领域，而如果两个物品属于很多用户的兴趣列表，那么它们就可能属于同一个领域，因而有很大的相似度。 

和UserCF算法类似，用ItemCF算法计算物品相似度时也可以首先建立用户—物品**倒排表**（即 对每个用户建立一个包含他喜欢的物品的列表），然后对于每个用户，将他物品列表中的物品两两在共现矩阵C中加1。详细代码如下所示：

In [ ]:
def ItemSimilarity(train): 
    #calculate co-rated users between items 
    C = dict() 
    N = dict() 
    for u, items in train.items(): 
        for i in users: 
            N[i] += 1 
            for j in users: 
                if i == j: 
                    continue 
                C[i][j] += 1
#calculate finial similarity matrix W 
    W = dict() 
    for i,related_items in C.items(): 
        for j, cij in related_items.items(): 
            W[u][v] = cij / math.sqrt(N[i] * N[j]) 
    return W

下图是一个根据上面的程序计算物品相似度的简单例子。![jupyter](./recommend-7.png)图中最左边是输入的用户行为记录，每一行代表一个用户感兴趣的物品集。然后，对于每个物品集合，我们将里面的物品两两加一，得到一个矩阵。最终将这些矩阵相加得到上面的C矩阵。其中C [i][j]记录了同时喜欢物品i和物品j的用户数。最后，将C矩阵归一化可以得到物品之间的余弦相似度矩阵W。

在得到物品之间的相似度后，ItemCF通过如下公式**计算用户u对一个物品j的兴趣**：$$P_{uj}=\sum_{i\in N(u)\cap S(j,K)}w_{ji}r_{ui}$$ 这里**$N(u)$是用户喜欢的物品的集合，$S(j, K)$是和物品i最相似的K个物品的集合，$w_{ji}$是物品j和i的相似度，$r_{ui}$是用户u对物品i的兴趣。（对于隐反馈数据集，如果用户u对物品i有过行为， 即可令rui=1。）该公式的含义是，和用户历史上感兴趣的物品越相似的物品，越有可能在用户的推荐列表中获得比较高的排名。该公式的实现代码如下所示。

In [ ]:
def Recommendation(train, user_id, W, K): 
    rank = dict() 
    ru = train[user_id] 
    for i,pi in ru.items(): 
        for j, wj in sorted(W[i].items(),key=itemgetter(1), reverse=True) [0:K]: 
            if j in ru: 
                continue 
            rank[j] += pi * wj 
    return rank

下图是一个基于物品推荐的简单例子。![jupyter](./recommend-8.png)该例子中，用户喜欢《C++ Primer中文版》和《编 程之美》两本书。然后ItemCF会为这两本书分别找出和它们最相似的3本书，然后根据公式的定义计算用户对每本书的感兴趣程度。比如，ItemCF给用户推荐《算法导论》，是因为这本书和《C++ Primer中文版》相似，相似度为0.4，而且这本书也和《编程之美》相似，相似度是0.5。考虑到用户对《C++ Primer中文版》的兴趣度是1.3，对《编程之美》的兴趣度是 0.9，那么用户对《算法导论》的兴趣度就是1.3 × 0.4 + 0.9×0.5 = 0.97。

#### 用户活跃度对物品相似度的影响 
从前面的讨论可以看到，在协同过滤中两个物品产生相似度是因为它们共同出现在很多用户的兴趣列表中。换句话说，每个用户的兴趣列表都对物品的相似度产生贡献。那么，是不是每个用户的贡献都相同呢？ 

假设有这么一个用户，他是开书店的，并且买了当当网上80%的书准备用来自己卖。那么，他的购物车里包含当当网80%的书。假设当当网有100万本书，也就是说他买了80万本。从前面对ItemCF的讨论可以看到，这意味着因为存在这么一个用户，有80万本书两两之间就产生了相似度，也就是说，内存里即将诞生一个80万乘80万的稠密矩阵。 另外可以看到，这个用户虽然活跃，但是买这些书并非都是出于自身的兴趣，而且这些书覆盖了当当网图书的很多领域，所以这个用户对于他所购买书的两两相似度的贡献应该远远小于一个只买了十几本自己喜欢的书的文学青年。 

John S. Breese提出了一个称为**IUF**（Inverse User Frequence），即**用户活跃度对数的倒数的参数**，他也认为活跃用户对物品相似度的贡献应该小于不活跃的用户，他提出应该增加IUF参数来修正物品相似度的计算公式：$$w_{ij}=\frac {\sum_{u\in N(i)\cap N(j)}\frac{1}{log(1+|N(u)|}}{\sqrt{|N(i)||N(j)|}}$$

当然，上面的公式只是对活跃用户做了一种**软性的惩罚**，但对于很多过于活跃的用户，比如上面那位买了当当网80%图书的用户，为了避免相似度矩阵过于稠密，我们在实际计算中一般直接忽略他的兴趣列表，而不将其纳入到相似度计算的数据集中。

In [ ]:
def ItemSimilarity(train): 
    #calculate co-rated users between items 
    C = dict() 
    N = dict() 
    for u, items in train.items(): 
        for i in users: 
            N[i] += 1 
            for j in users: 
                if i == j: 
                    continue 
                C[i][j] += 1 / math.log(1 + len(items) * 1.0) 
    #calculate finial similarity matrix W 
    W = dict() 
    for i,related_items in C.items(): 
        for j, cij in related_items.items(): 
            W[u][v] = cij / math.sqrt(N[i] * N[j]) 
    return W

ItemCF-IUF在准确率和召回率两个指标上和ItemCF相近，但ItemCF-IUF明显 提高了推荐结果的覆盖率，降低了推荐结果的流行度。从这个意义上说，ItemCF-IUF确实改 进了ItemCF的综合性能。

#### 物品相似度的归一化 
Karypis在研究中发现如果将ItemCF的相似度矩阵按最大值归一化，可以提高推荐的准确率。 其研究表明，如果已经得到了物品相似度矩阵w，那么可以用如下公式得到归一化之后的相似度矩阵$w^{'}=\frac{w_{ij}}{max_jw_{ij}}$，其实，归一化的好处不仅仅在于增加推荐的准确度，它还可以提高推荐的覆盖率和多样性。 一般来说，物品总是属于很多不同的类，每一类中的物品联系比较紧密。

#### 基于物品的协同过滤算法代码实现

In [ ]:
#-*- coding: utf-8 -*-
'''
Created on 2015-06-22
@author: Lockvictor
'''
import sys
import random
import math
import os
from operator import itemgetter
from collections import defaultdict

random.seed(0)

'''
users.dat 数据集 
用户id 用户性别 用户年龄 用户职业 用户所在地邮编
1::F::1::10::48067
2::M::56::16::70072
3::M::25::15::55117
movies.dat 数据集
电影id 电影名称 电影类型 
250::Heavyweights (1994)::Children's|Comedy
251::Hunted, The (1995)::Action
252::I.Q. (1994)::Comedy|Romance
ratings.dat 数据集
用户id 电影id 用户评分  时间戳
157::3519::4::1034355415
157::2571::5::977247494
157::300::3::977248224
'''

'''基于物品协同过滤算法'''
class ItemBasedCF(object):
    ''' TopN recommendation - Item Based Collaborative Filtering '''

    def __init__(self):
        self.trainset = {} # 存储训练集
        self.testset = {} # 存储测试集

        self.n_sim_movie = 20 # 定义相似电影数20
        self.n_rec_movie = 10 # 定义推荐电影数10

        self.movie_sim_mat = {} # 存储电影相似矩阵
        self.movie_popular = {}
        self.movie_count = 0

        print('Similar movie number = %d' % self.n_sim_movie, file=sys.stderr)
        print('Recommended movie number = %d' %
              self.n_rec_movie, file=sys.stderr)

    @staticmethod
    def loadfile(filename):
        ''' 只读的方式打开文件 '''
        fp = open(filename, 'r')
        # enumerate()为枚举，i为行号从0开始，line为值
        for i, line in enumerate(fp):
            # yield 迭代去下一个值，类似next()
            # line.strip()用于去除字符串头尾指定的字符。
            yield line.strip('\r\n')
            if i % 100000 == 0:
                print ('loading %s(%s)' % (filename, i), file=sys.stderr)
        fp.close()
        print ('load %s succ' % filename, file=sys.stderr)

    def generate_dataset(self, filename, pivot=0.7):
        ''' 加载数据并且将数据划分为训练集和测试集'''
        trainset_len = 0
        testset_len = 0

        for line in self.loadfile(filename):
            user, movie, rating, _ = line.split('::')
            # 按照pivot=0.7 比例划分
            if random.random() < pivot:
                '''训练集 数据为 {userid,{moveiesid,rating。。。。}}'''
                self.trainset.setdefault(user, {})
                self.trainset[user][movie] = int(rating)
                trainset_len += 1
            else:
                '''测试集 数据为 {userid,{moveiesid,rating....}}'''
                self.testset.setdefault(user, {})
                self.testset[user][movie] = int(rating)
                testset_len += 1

        print ('划分数据集成功', file=sys.stderr)
        print ('训练集 = %s' % trainset_len, file=sys.stderr)
        print ('测试集 = %s' % testset_len, file=sys.stderr)

    def calc_movie_sim(self):
        ''' 记录电影被观看过的次数，从而反映出电影的流行度'''
        print('counting movies number and popularity...', file=sys.stderr)
        # 计算电影的流行度 实质是一个电影被多少用户操作过（即看过，并有评分） 以下数据虚构
        '''{'914': 23, '3408': 12, '2355': 4, '1197': 12, '2804': 31, '594': 12  .....}'''
        for user, movies in self.trainset.items():
            for movie in movies:
                # count item popularity
                if movie not in self.movie_popular:
                    # 如果电影第一次出现 则置为0 假如到字典中。
                    self.movie_popular[movie] = 0
                self.movie_popular[movie] += 1

        print('count movies number and popularity succ', file=sys.stderr)

        # 计算总共被看过的电影数
        self.movie_count = len(self.movie_popular)
        print('total movie number = %d' % self.movie_count, file=sys.stderr)

        # 根据用户使用习惯 构建物品相似度
        itemsim_mat = self.movie_sim_mat
        print('building co-rated users matrix...', file=sys.stderr)
        '''以下为数据格式，通过for循环依次遍历训练集，找到两个电影之间，如果被一个人同时看过，则累加一'''
        #{'914': defaultdict( <class 'int'>, {'3408': 1, '2355': 1  , '1197': 1, '2804': 1, '594': 1, '919': 1})}
        for user, movies in self.trainset.items():
            for m1 in movies:
                itemsim_mat.setdefault(m1, defaultdict(int))
                for m2 in movies:
                    if m1 == m2:
                        continue
                    itemsim_mat[m1][m2] += 1

        print('build co-rated users matrix succ', file=sys.stderr)

        # calculate similarity matrix
        print('calculating movie similarity matrix...', file=sys.stderr)
        simfactor_count = 0
        PRINT_STEP = 2000000

        # 计算用户相似度矩阵
            # 先取得特定用户
        for m1, related_movies in itemsim_mat.items():
            for m2, count in related_movies.items():
                # 以下公式为 两个a,b电影共同被喜欢的用户数/ 根号下（喜欢电影a的用户数 乘 喜欢电影 b的用户数）
                itemsim_mat[m1][m2] = count / math.sqrt(
                    self.movie_popular[m1] * self.movie_popular[m2])
                simfactor_count += 1
                if simfactor_count % PRINT_STEP == 0:
                    print('calculating movie similarity factor(%d)' %
                          simfactor_count, file=sys.stderr)

        print('calculate movie similarity matrix(similarity factor) succ',
              file=sys.stderr)
        print('Total similarity factor number = %d' %
              simfactor_count, file=sys.stderr)
    '''找到K个相似电影 并推荐N个电影'''
    def recommend(self, user):

        K = self.n_sim_movie
        N = self.n_rec_movie
        rank = {}
        watched_movies = self.trainset[user]

        for movie, rating in watched_movies.items():
            # 对于用户看过的每个电影 都找出其相似度最高的前K个电影
            for related_movie, similarity_factor in sorted(self.movie_sim_mat[movie].items(),
                                                           key=itemgetter(1), reverse=True)[:K]:
                if related_movie in watched_movies:
                    continue
                rank.setdefault(related_movie, 0)
                # 假如评分权重，
                rank[related_movie] += similarity_factor * rating
        # 返回综合评分最高的N个电影
        return sorted(rank.items(), key=itemgetter(1), reverse=True)[:N]
    # 计算准确率，召回率，覆盖率，流行度
    def evaluate(self):
        ''' print evaluation result: precision, recall, coverage and popularity '''
        print('Evaluation start...', file=sys.stderr)

        N = self.n_rec_movie
        #  varables for precision and recall
        hit = 0
        rec_count = 0
        test_count = 0
        # varables for coverage
        all_rec_movies = set()
        # varables for popularity
        popular_sum = 0

        for i, user in enumerate(self.trainset):
            if i % 500 == 0:
                print ('recommended for %d users' % i, file=sys.stderr)
            test_movies = self.testset.get(user, {})
            rec_movies = self.recommend(user)
            for movie, _ in rec_movies:
                if movie in test_movies:
                    hit += 1
                all_rec_movies.add(movie)
                popular_sum += math.log(1 + self.movie_popular[movie])
            rec_count += N
            test_count += len(test_movies)
        # 准确率 = 推荐中的电影/总推荐的电影
        precision = hit / (1.0 * rec_count)
        # 召回率 = 推荐中的电影/测试集中所有电影数目
        recall = hit / (1.0 * test_count)
        coverage = len(all_rec_movies) / (1.0 * self.movie_count)
        popularity = popular_sum / (1.0 * rec_count)

        print ('precision=%.4f\trecall=%.4f\tcoverage=%.4f\tpopularity=%.4f' %
               (precision, recall, coverage, popularity), file=sys.stderr)


if __name__ == '__main__':
    ratingfile = os.path.join('Recomment_System', 'ratings.dat')
    itemcf = ItemBasedCF()
    itemcf.generate_dataset(ratingfile)
    itemcf.calc_movie_sim()
    print(itemcf.recommend('1'))
    #itemcf.evaluate()

### UserCF 和 ItemCF 的综合比较
UserCF是推荐系统领域较为古老的算法，1992年就已经在电子邮件的个性化推荐系统 Tapestry中得到了应用，1994年被GroupLens用来实现新闻的个性化推荐，后来被著名的**文章分享网站Digg**用来给用户推荐个性化的网络文章。ItemCF则是相对比较新的算法，在著名的电子商务网站**亚马逊**和DVD租赁网站Netflix中得到了广泛应用。

那么，**为什么Digg使用UserCF，而亚马逊网使用ItemCF呢？**

首先回顾一下UserCF算法和ItemCF算法的推荐原理。UserCF给用户推荐那些和他有共同兴趣爱好的用户喜欢的物品，而ItemCF给用户推荐那些和他之前喜欢的物品类似的物品。从这个算法的原理可以看到，UserCF的推荐结果着重于反映和用户兴趣相似的小群体的热点，而 ItemCF的推荐结果着重于维系用户的历史兴趣。换句话说，UserCF的推荐更社会化，反映了用户所在的小型兴趣群体中物品的热门程度，而ItemCF的推荐更加个性化，反映了用户自己的兴趣传承。 

在新闻网站中，用户的兴趣不是特别细化，绝大多数用户都喜欢看热门的新闻。即使是个性 化，也是比较粗粒度的，比如有些用户喜欢体育新闻，有些喜欢社会新闻，而特别细粒度的个性化一般不存在。比方说，很少有用户只看某个话题的新闻，主要是因为这个话题不 可能保证每天都有新的消息，而这个用户却是每天都要看新闻的。因此，个性化新闻推荐更 加强调抓住新闻热点，热门程度和时效性是个性化新闻推荐的重点，而个性化相对于这两点 略显次要。因此，UserCF可以给用户推荐和他有相似爱好的一群其他用户今天都在看的新 闻，这样在抓住热点和时效性的同时，保证了一定程度的个性化。这是Digg在新闻推荐中使用UserCF的最重要原因。 

UserCF适合用于新闻推荐的另一个原因是从技术角度考量的。因为作为一种物品，新闻的更 新非常快，每时每刻都有新内容出现，而ItemCF需要维护一张物品相关度的表，如果物品更 新很快，那么这张表也需要很快更新，这在技术上很难实现。绝大多数物品相关度表都只能 做到一天一次更新，这在新闻领域是不可以接受的。而UserCF只需要用户相似性表，虽然 UserCF对于新用户也需要更新相似度表，但在新闻网站中，物品的更新速度远远快于新用户 的加入速度，而且对于新用户，完全可以给他推荐最热门的新闻，因此UserCF显然是利大于 弊。

但是，在图书、电子商务和电影网站，比如亚马逊、豆瓣、Netflix中，ItemCF则能极大地发 挥优势。首先，在这些网站中，用户的兴趣是比较固定和持久的。一个技术人员可能都是在 购买技术方面的书，而且他们对书的热门程度并不是那么敏感，事实上越是资深的技术人 员，他们看的书就越可能不热门。此外，这些系统中的用户大都不太需要流行度来辅助他们 判断一个物品的好坏，而是可以通过自己熟悉领域的知识自己判断物品的质量。因此，这些 网站中个性化推荐的任务是帮助用户发现和他研究领域相关的物品。因此，ItemCF算法成为 了这些网站的首选算法。此外，这些网站的物品更新速度不会特别快，一天一次更新物品相 似度矩阵对它们来说不会造成太大的损失，是可以接受的。 

同时，从技术上考虑，UserCF需要维护一个用户相似度的矩阵，而ItemCF需要维护一个物品 相似度矩阵。从存储的角度说，如果用户很多，那么维护用户兴趣相似度矩阵需要很大的空 间，同理，如果物品很多，那么维护物品相似度矩阵代价较大。 

在早期的研究中，大部分研究人员都是让少量的用户对大量的物品进行评价，然后研究用户 兴趣的模式。那么，对于他们来说，因为用户很少，计算用户兴趣相似度是最快也是最简单 的方法。但在实际的互联网中，用户数目往往非常庞大，而在图书、电子商务网站中，物品 的数目则是比较少的。此外，物品的相似度相对于用户的兴趣一般比较稳定，因此使用 ItemCF是比较好的选择。当然，新闻网站是个例外，在那儿，物品的相似度变化很快，物品 数目庞大，相反用户兴趣则相对固定（都是喜欢看热门的），所以新闻网站的个性化推荐使 用UserCF算法的更多。
![jupyter](./recommend-9.png)

## 2.5 隐语义模型 
自从Netflix Prize比赛举办以来，**LFM（latent factor model）隐语义模型**逐渐成为推荐系统领域耳熟能详的名词。其实该算法最早在文本挖掘领域被提出，用于找到文本的隐含语义。相关的名词有**LSI、pLSA、LDA和Topic Model**。本节将对隐含语义模型在Top-N推荐中的应用 进行详细介绍，并通过实际的数据评测该模型。 
### 基础算法 
隐语义模型是最近几年推荐系统领域最为热门的研究话题，它的核心思想是通过**隐含特征**(latent factor)联系用户兴趣和物品。

首先通过一个例子来理解一下这个模型。两个用户在豆瓣的读书列表，用户A的兴趣涉及侦探小说、科普图书以及一些计算机技术书，而用户B的兴趣比较集中在数学和机器学习方面。 那么**如何给A和B推荐图书**呢？ 

    对于UserCF，首先需要找到和他们看了同样书的其他用户（兴趣相似的用户）， 然后给他们推荐那些用户喜欢的其他书。 
    对于ItemCF，需要给他们推荐和他们已经看的书相似的书，比如作者B看了很多关于数据挖掘的书，可以给他推荐机器学习或者模式识别方面的书。 
    
还有一种方法，可以对书和物品的兴趣进行分类。对于某个用户，首先得到他的兴趣分类， 然后从分类中挑选他可能喜欢的物品。 这就是隐语义模型，依据“兴趣”这一隐含特征将用户与物品进行连接，需要说明的是此处的**“兴趣”**其实是对物品类型的一个分类而已。

**隐语义的数学理解**：

如下图所示，R矩阵是用户对物品的偏好信息（$R_{ij}$表示的是用户i对物品j的兴趣度），P矩阵是用户对各物品类别的一个偏好信息（$P_{ij}$表示的是user i对class j的兴趣度），Q矩阵是各物品所归属的物品类别的信息（$Q_{ij}$表示的是item j在class i中的权重）。**隐语义模型就是要将矩阵R分解为矩阵P和矩阵Q的乘积，即通过矩阵中的物品类别(class)将用户user和物品item联系起来**。实际上我们需要根据用户当前的物品偏好信息R进行计算，从而得到对应的矩阵P和矩阵Q。![jupyter](./recommend-10.jpg)
 
总结一下，这个基于兴趣分类的方法大概**需要解决3个问题**。 
 
如何给物品进行分类？ 
 
如何确定用户对哪些类的物品感兴趣，以及感兴趣的程度？ 
 
对于一个给定的类，选择哪些属于这个类的物品推荐给用户，以及如何确定这些 物品在一个类中的权重？ 

对于第一个问题的简单解决方案是找编辑给物品分类。以图书为例，每本书出版时，编辑都会给书一个分类。为了给图书分类，出版界普遍遵循**中国图书分类法**。但是，即使有很系统的分类体系，编辑给出的分类仍然具有以下缺点：

1）编辑的意见不能代表各种用户的意见。

2）编辑很难控制分类的粒度。我们知道分类是有不同粒度的，《数据挖掘导论》在粗粒度的分类中可能属于计算机技术，但在细粒度的分类中可能属于数据挖掘。 

3）编辑很难给一个物品多个分类。有的书不仅属于一个类，而是可能属于很多的类。

4）编辑很难给出多维度的分类。比如按照作 者分类、按照译者分类、按照出版社分类。

5）编辑很难决定一个物品在某一个分类中的权重。辑就很难给出一个准确的数字来表示。

为了解决上面的问题，研究人员提出：**为什么我们不从数据出发，自动地找到那些类，然后进行个性化推荐**？**隐含语义分析技术**（latent variable analysis）因为采取基于用户行为统计的**自动聚类**，较好地解决了上面提出的5个问题。

隐含语义分析技术从诞生到今天产生了很多著名的模型和方法，其中和该技术相关且耳熟能详的名词有pLSA、LDA、隐含类别模型（latent class model）、隐含主题模型（latent topic model）、矩阵分解（matrix factorization）。这些技术和方法在本质上是相通的，其中很多都可以用于个性化推荐系统。本章将以LFM为例介绍隐含语义分析技术在推荐系统中的 应用。 

按照隐语义数学理解，LFM通如下公式**计算用户u对物品i的兴趣**：$$Preference(u,i)=r_{ui}=p_u^Tq_i=\sum_{k=1}^Kp_{u,k}q_{k,i}$$

隐语义模型会把物品分成K个类型，$p_{u,k},q_{k,i}$是模型的参数，是根据经验和业务知识进行反复尝试决定的，也即是从训练集中训练出来的，前者表示用户u对于第k个分类的喜爱程度，后者表示物品i属于第k个分类的权重。

现在我们讨论下如何计算矩阵P和矩阵Q中的参数值。一般做法就是最优化损失函数来求参数。
损失函数如下所示：![jupyter](./recommend-11.jpg)

上式中的![jupyter](./recommend-12.png)是用来防止过拟合的正则化项，λ需要根据具体应用场景反复实验得到。损失函数的意义是用户u对物品i的真实喜爱程度与推算出来的喜爱程度的**均方根误差**，通俗来说就是真实的喜爱程度与推算的喜爱程度的误差，要使模型最合理当然就是使这个误差达到最小值。公式中最后两项是**惩罚因子**，用来防止分类数取得过大而使误 差减少的不合理做法的发生，λ参数是一个常数，需要根据经验和业务知识进行反复尝试决定的。

损失函数的优化使用随机梯度下降算法：

1）对两组未知数求偏导数
![jupyter](./recommend-13.jpg)

2）根据随机梯度下降法得到递推公式![jupyter](./recommend-14.jpg)

其中**α是在梯度下降的过程中的步长(也可以称作学习速率)**，这个值不宜过大也不宜过小，过大会产生震荡而导致很难求得最小值，过小会造成计算速度下降，需要经过试验得到最合适的值。最终会求得每个用户对于每个隐分类的喜爱程度矩阵P和每个物品与每个隐分类的匹配程度矩阵Q。在用户对物品的偏好信息矩阵R 中，通过迭代可以求得每个用户对每个物品的喜爱程度,选取喜爱程度最高而且用户没有反馈过的物品进行推荐。

在隐语义模型中，重要的参数有以下4个：

1）隐分类的个数F；

2）梯度下降过程中的步长(学习速率)α；

3）损失函数中的惩罚因子λ；

4）正反馈样本数和负反馈样本数的比例ratio；

这四项参数需要在试验过程中获得最合适的值.1）3）4）这三项需要根据推荐系统的准确率、召回率、覆盖率及流行度作为参考, 而2）步长α要参考模型的训练效率。

**隐语义模型的样本问题**

隐语义模型在显性反馈数据（也就是评分数据）上能解决评分预测问题并达到了很好的精度。不过推荐系统主要讨论的是隐性反馈数据集，这种数据集的特点是只有正样本（用户喜欢什么物品），而没有负样本（用户对什么物品不感兴趣）。那么，在隐性反馈数据集上应用隐语义模型解决推荐的第一个关键问题就是如何给每个用户生成负样本。可能的方法如下：

1）对每个用户，用他没有过行为的物品作为负样本。这样做，负样本太多，正负样本相差悬殊。

2）对每个用户，用他没有过行为的物品均匀采样出一些物品作为负样本。

**3）类似2），但是保证正负样本数目相当。**

4）类似2），但是采样时，偏重采样不热门的物品。

Rong Pan的文章表示，第三种好于第二种，而第二种好于第四种。

通过2011年的KDD Cup的Yahoo! Music推荐系统比赛，我们发现对负样本采样时应该 遵循以下原则。

1）对于每个用户，保证正负样本均衡。

2）对每个用户采样负样本时，要选取那些很热门，但用户却没有行为的物品，这更加表示用户对这个物品不感兴趣。因为对于冷门的物品，用户可能压根没在网站发现这个物品，所以谈不上是否感兴趣。

下面的Python代码实现了负样本采样过程：

In [ ]:
def RandomSelectNegativeSample(items): 
    ret = dict() 
    for i in items.keys(): 
        ret[i] = 1 
    n = 0 
    for i in range(0, len(items) * 3): 
        item = items_pool[random.randint(0, len(items_pool) - 1)] 
        if item in ret: 
            continue 
        ret[item] = 0 
        n += 1 
        if n > len(items): 
            break 
    return ret

In [ ]:
##负样本采样过程
##正样本：用户喜欢的物品； 
##负样本：用户不感兴趣的物品。
import random

items = {'a':1,'b':1,'c':1}  # 用户已经有过行为的物品的集合，即用户喜欢的物品列表，即正样本
items_pool = ['a','b','c','d','e','f','g','h','i']
'''
ret = dict()  # 用来存储正样本和负样本，即用户喜欢的和不感兴趣的物品都将存储在这里

for i in items.keys(): 
    ret.update({i:1})  # 将用户已经有过行为的物品，添加到ret字典中，并标注成正样本

items_pool = ['a','b','c','d','e','f','g','h','i']
n = 0  # 计数，计算已经找到的负样本的数目
for i in range(0,len(items)*3):  # 当然，也有可能循环len(items)*3次后，找到的负样本还是不如正样本多，但这毕竟是小概率事件
    item = items_pool[random.randint(1,len(items_pool) - 1)]  # items_pool 是候选物品的列表，包括正样本和负样本。（注意！但不是所有物品，因为除了正样本后，不止有负样本，还有热门物品中，用户不感兴趣的物品）。 换言之，items_pool[]中不包括热门物品中，用户不感兴趣的物品
    if item in ret.keys():
        continue
    ret.update({item:0})
    n += 1
    if n >= len(items):
        break
'''
ret=RandomSelectNegativeSample(items)
for item,ret in ret.items():
    print(item,'correspond to ',ret)

**优缺点分析**

隐语义模型在实际使用中有一个困难，那就是它**很难实现实时推荐**。经典的隐语义模型每次训练时都需要扫描所有的用户行为记录，这样才能计算出用户对于每个隐分类的喜爱程度矩阵P和每个物品与每个隐分类的匹配程度矩阵Q。而且隐语义模型的训练需要在用户行为记录上反复迭代才能获得比较好的性能，因此LFM的每次训练都很耗时，一般在实际应用中只能每天训练一次，并且计算出所有用户的推荐结果。从而隐语义模型不能因为用户行为的变化实时地调整推荐结果 来满足用户最近的行为。

### 隐语义模型LFM算法python实现

In [ ]:
# -*- coding: utf-8 -*-
'''Created on 2017年9月15日@author: Jason.F''' 
from math import exp
import pandas as pd
import numpy as np
import pickle
import time
##from numpy import rank  

def getUserNegativeItem(frame, userID):    
    '''    
    获取用户负反馈物品：热门但是用户没有进行过评分 与正反馈数量相等    
    :param frame: ratings数据    
    :param userID:用户ID    
    :return: 负反馈物品    
    '''    
    userItemlist = list(set(frame[frame['userid'] == userID]['itemid']))                       #用户评分过的物品    
    otherItemList = [item for item in set(frame['itemid'].values) if item not in userItemlist] #用户没有评分的物品    
    itemCount = [len(frame[frame['itemid'] == item]['userid']) for item in otherItemList]      #物品热门程度    
    series = pd.Series(itemCount, index=otherItemList)    
    series = series.sort_values(ascending=False)[:len(userItemlist)]                            #获取正反馈物品数量的负反馈物品    
    negativeItemList = list(series.index)    
    return negativeItemList  

def getUserPositiveItem(frame, userID):    
    '''    
    获取用户正反馈物品：用户评分过的物品    
    :param frame: ratings数据    
    :param userID: 用户ID    
    :return: 正反馈物品    
    '''    
    series = frame[frame['userid'] == userID]['itemid']    
    positiveItemList = list(series.values)    
    return positiveItemList  

def initUserItem(frame, userID=1):    
    '''    
    初始化用户正负反馈物品,正反馈标签为1,负反馈为0    
    :param frame: ratings数据    
    :param userID: 用户ID    
    :return: 正负反馈物品字典    
    '''    
    positiveItem = getUserPositiveItem(frame, userID)   
    negativeItem = getUserNegativeItem(frame, userID)    
    itemDict = {}    
    for item in positiveItem: itemDict[item] = 1    
    for item in negativeItem: itemDict[item] = 0    
    return itemDict 

def initUserItemPool(frame,userID):    
    '''    
    初始化目标用户样本    
    :param userID:目标用户    
    :return:    
    '''    
    userItem = []    
    for id in userID:        
        itemDict = initUserItem(frame, userID=id)        
        userItem.append({id:itemDict})    
    return userItem 

def initPara(userID, itemID, classCount):    
    '''    初始化参数q,p矩阵, 随机    :param userCount:用户ID    :param itemCount:物品ID    :param classCount: 隐类数量    :return: 参数p,q    '''    
    arrayp = np.random.rand(len(userID), classCount) #构造p矩阵，[0,1]内随机值    
    arrayq = np.random.rand(classCount, len(itemID)) #构造q矩阵，[0,1]内随机值    
    p = pd.DataFrame(arrayp, columns=range(0,classCount), index=userID)    
    q = pd.DataFrame(arrayq, columns=itemID, index=range(0,classCount))        
    return p,q 

def initModel(frame, classCount):    
    '''    初始化模型：参数p,q,样本数据    :param frame: 源数据    :param classCount: 隐类数量    :return:    '''    
    userID = list(set(frame['userid'].values))    
    itemID = list(set(frame['itemid'].values))    
    p, q = initPara(userID, itemID, classCount)#初始化p、q矩阵    
    userItem = initUserItemPool(frame,userID)#建立用户-物品对应关系    
    return p, q, userItem 

def latenFactorModel(frame, classCount, iterCount, alpha, lamda):    
    '''    隐语义模型计算参数p,q    :param frame: 源数据    :param classCount: 隐类数量    :param iterCount: 迭代次数    :param alpha: 步长    :param lamda: 正则化参数    :return: 参数p,q    '''    
    p, q, userItem = initModel(frame, classCount)    
    for step in range(0, iterCount):        
        for user in userItem:            
            for userID, samples in user.items():                
                for itemID, rui in samples.items():                    
                    eui = rui - lfmPredict(p, q, userID, itemID)                    
                    for f in range(0, classCount):                        
                        print('step %d user %d class %d' % (step, userID, f))                        
                        p[f][userID] += alpha * (eui * q[itemID][f] - lamda * p[f][userID])                        
                        q[itemID][f] += alpha * (eui * p[f][userID] - lamda * q[itemID][f])        
        alpha *= 0.9
    return p, q 

def sigmod(x):    
    '''    单位阶跃函数,将兴趣度限定在[0,1]范围内    :param x: 兴趣度    :return: 兴趣度    '''    
    y = 1.0/(1+exp(-x))    
    return y  

def lfmPredict(p, q, userID, itemID):    
    '''    利用参数p,q预测目标用户对目标物品的兴趣度    :param p: 用户兴趣和隐类的关系    :param q: 隐类和物品的关系    :param userID: 目标用户    :param itemID: 目标物品    :return: 预测兴趣度    '''    
    p = np.mat(p.ix[userID].values)    
    q = np.mat(q[itemID].values).T    
    r = (p * q).sum()    
    r = sigmod(r)    
    return r 

def recommend(frame, userID, p, q, TopN=10):    
    '''    推荐TopN个物品给目标用户    :param frame: 源数据    :param userID: 目标用户    :param p: 用户兴趣和隐类的关系    :param q: 隐类和物品的关系    :param TopN: 推荐数量    :return: 推荐物品    '''    
    userItemlist = list(set(frame[frame['userid'] == userID]['itemid']))    
    otherItemList = [item for item in set(frame['itemid'].values) if item not in userItemlist]    
    predictList = [lfmPredict(p, q, userID, itemID) for itemID in otherItemList]    
    series = pd.Series(predictList, index=otherItemList)    
    series = series.sort_values(ascending=False)[:TopN]    
    return series 

def Recall(df_test,p,q):#召回率    
    hit=0    
    all=0    
    df_userid=df_test['userid']    
    df_userid=df_userid.drop_duplicates()    
    for userid in df_userid:        
        pre_item=recommend(df_test, userid, p, q)        
        df_user_item=df_test.loc[df_test['userid'] == userid]        
        true_item=df_user_item['itemid']        
        for itemid,prob in pre_item.items():            
            if itemid in true_item:                
                hit+=1        
        all+=len(true_item)    
    return hit/(all*1.0) 

def Precision(df_test,p,q):    
    hit=0    
    all=0    
    df_userid=df_test['userid']    
    df_userid=df_userid.drop_duplicates()    
    for userid in df_userid:        
        pre_item=recommend(df_test, userid, p, q)        
        df_user_item=df_test.loc[df_test['userid'] == userid]        
        true_item=df_user_item['itemid']        
        for itemid,prob in pre_item.items():            
            if itemid in true_item:                
                hit+=1        
        all+=len(pre_item)    
    return hit/(all*1.0)    

def Coverage(df_test,p,q):#覆盖率        
    df_itemid=df_test['itemid']    
    df_itemid=df_itemid.drop_duplicates()    
    rec_items=set()#推荐的物品总数    
    df_userid=df_test['userid']    
    df_userid=df_userid.drop_duplicates()    
    for userid in df_userid:        
        pre_item=recommend(df_test, userid, p, q)        
        for itemid,prob in pre_item.items():            
            rec_items.add(itemid)    
    return len(rec_items)/(len(df_itemid)*1.0)

if __name__ == '__main__':        
    start = time.clock()          
    #导入数据    
    df_sample = pd.read_csv("ratings.csv",names=['userid','itemid','ratings','time'],header=0)      
    #模型训练    
    p, q = latenFactorModel(df_sample,5, 3, 0.02, 0.01 )    
    #模型评估    
    df_test=df_sample.sample(frac=0.2)#抽20%来测试    
    print (Recall(df_test,p,q))#召回率    
    print (Precision(df_test,p,q))#准确率
    print (Coverage(df_test,p,q))#覆盖率
    end = time.clock()   

    print('finish all in %s' % str(end - start)) 


## 基于图的模型
首先需要将用户行为数据表示成图的形式。用户行为数据是由一系列二元组组成的，其中每个二元组(u, i)表示用户u对物品i产生过行为。这种数 据集很容易用一个二分图表示。

图有用户节点、物品节点。如果有过行为，则用户和物品之间有连线。模型示意图如下：![jupyter](./recommend-15.png)

### 基于图的推荐算法
将个性化推荐放在二分图模型中，那么给用户u推荐物品任务可以转化为度量用户顶点$V_u$和与$V_u$没有边直接相连的物品节点在图上的**相关度**，相关度越高的在推荐列表中越靠前。

图中顶点的相关度主要取决与以下因素： 

1）两个顶点之间路径数 

2）两个顶点之间路径长度 

3）两个顶点之间路径经过的顶点 

而相关性高的顶点一般有如下特性： 

1）两个顶点有很多路径相连 

2）连接两个顶点之间的路径长度比较短 

3）连接两个顶点之间的路径不会经过出度较大的顶点

下面详细介绍**基于随机游走的PersonalRank算法**。 

假设给用户u进行个性化推荐，从图中用户u对应的节点$V_u$开始游走，游走到一个节点时，首先按照概率alpha决定是否继续游走，还是停止这次游走并从$V_u$节点开始重新游走。如果决定继续游走，那么就从当前节点指向的节点中按照**均匀分布**随机选择一个节点作为下次经过的节点，这样经过很多次的随机游走后，每个物品节点被访问到的概率就会收敛到一个数。最终推荐列表中物品的权重就是物品节点的访问概率。 
迭代公式如下：![jupyter](./recommend-16.png)公式中PR(i)表示物品i的访问概率(也即是物品i的权重)，out(i)表示物品节点i的出度。alpha决定继续访问的概率。

以下面的二分图为例，实现PersonalRank算法![jupyter](./recommend-17.png)

In [1]:
#coding:utf-8
import time 
def PersonalRank(G,alpha,root,max_depth):    
    rank=dict()    
    rank={x:0 for x in G.keys()}    
    rank[root]=1    
    #开始迭代    
    begin=time.time()    
    for k in range(max_depth):        
        tmp={x:0 for x in G.keys()}        
        #取出节点i和他的出边尾节点集合ri        
        for i,ri in G.items():            
            #取节点i的出边的尾节点j以及边E(i,j)的权重wij,边的权重都为1，归一化后就是1/len(ri)            
            for j,wij in ri.items():                
                tmp[j]+=alpha*rank[i]/(1.0*len(ri))        
        tmp[root]+=(1-alpha)        
        rank=tmp    
    end=time.time()    
    print ('use_time',end-begin)    
    lst=sorted(rank.items(),key=lambda x:x[1],reverse=True)    
    for ele in lst:        
        print ("%s:%.3f, \t" %(ele[0],ele[1]))     
    return rank 

if __name__=='__main__':    
    alpha=0.8    
    G = {'A': {'a': 1, 'c': 1},         
         'B': {'a': 1, 'b': 1, 'c': 1, 'd': 1},        
         'C': {'c': 1, 'd': 1},         
         'a': {'A': 1, 'B': 1},         
         'b': {'B': 1},         
         'c': {'A': 1, 'B': 1, 'C': 1},         
         'd': {'B': 1, 'C': 1}}    
    PersonalRank(G,alpha,'b',50)


use_time 0.0
B:0.312, 	
b:0.262, 	
c:0.115, 	
a:0.089, 	
d:0.089, 	
A:0.066, 	
C:0.066, 	


上面算法在时间复杂度上有个明显的缺陷，每次为每个用户推荐时，都需要在整个用户物品二分图上进行迭代，直到整个图上每个节点收敛。这一过程**时间复杂度非常高**，不仅无法提高实时推荐，甚至离线生产推荐结果也很耗时。

为了解决时间复杂度过高问题，我们可以从矩阵角度出发，personalrank经过多次的迭代游走，使得各节点的重要度趋于稳定，实际上我们根据**状态转移矩阵**，经过一次矩阵运算就可以直接得到系统的稳态。上面迭代公式的矩阵表示形式为： ![jupyter](./recommend-18.png)其中r是个n维向量，每个元素代表一个节点的PR重要度，r0也是个n维向量，第i个位置上是1，其余元素均为0，上面迭代公式左边有个PR(j)和PR(i)，其中i是j一条入边。他们迭代最终的概率（权重）都在r中。我们就是要为第i个节点进行推荐。M是n阶转移矩阵： ![jupyter](./recommend-19.png)上式可以变形到：![jupyter](./recommend-20.png)这就相当于解线性方程组了.